# 🧪 ADK Application Testing

This notebook demonstrates how to test an ADK (Agent Development Kit) application.
It covers both local and remote testing, both with Agent Engine and Cloud Run.

> **Note**: This notebook assumes that the agent files are stored in the `app` folder. If your agent files are located in a different directory, please update all relevant file paths and references accordingly.

## Install dependencies

In [ ]:
!pip install google-cloud-aiplatform a2a-sdk --upgrade

### Import libraries

In [ ]:
import json
import uuid

import requests
import vertexai
from a2a.types import (
    Message,
    MessageSendParams,
    Part,
    Role,
    SendStreamingMessageRequest,
    TextPart,
)

### Initialize Vertex AI Client

In [ ]:
# Initialize the Vertex AI client
LOCATION = "us-central1"

client = vertexai.Client(
    location=LOCATION,
)

## If you are using Agent Engine
See more documentation at [Agent Engine Overview](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)

### Remote Testing

In [ ]:
# Set to None to auto-detect from ./deployment_metadata.json, or specify manually
# "projects/PROJECT_ID/locations/us-central1/reasoningEngines/ENGINE_ID"
REASONING_ENGINE_ID = None

if REASONING_ENGINE_ID is None:
    try:
        with open("../deployment_metadata.json") as f:
            metadata = json.load(f)
            REASONING_ENGINE_ID = metadata.get("remote_agent_engine_id")
    except (FileNotFoundError, json.JSONDecodeError):
        pass

print(f"Using REASONING_ENGINE_ID: {REASONING_ENGINE_ID}")
# Get the existing agent engine
remote_agent_engine = client.agent_engines.get(name=REASONING_ENGINE_ID)

In [ ]:
async for event in remote_agent_engine.async_stream_query(
    message="hi!", user_id="test"
):
    print(event)

In [ ]:
remote_agent_engine.register_feedback(
    feedback={
        "score": 5,
        "text": "Great response!",
        "invocation_id": "test-invocation-123",
        "user_id": "test",
    }
)

### Local Testing

You can import directly the AgentEngineApp class within your environment. 
To run the agent locally, follow these steps:
1. Make sure all required packages are installed in your environment
2. The recommended approach is to use the same virtual environment created by the 'uv' tool
3. You can set up this environment by running 'make install' from your agent's root directory
4. Then select this kernel (.venv folder in your project) in your Jupyter notebook to ensure all dependencies are available

In [ ]:
# Uncomment the following lines if you're not using the virtual environment created by uv
# import sys
# sys.path.append("../")

In [ ]:
from app.agent import root_agent
from app.agent_engine_app import AgentEngineApp

agent_engine = AgentEngineApp(agent=root_agent)

In [ ]:
async for event in agent_engine.async_stream_query(message="hi!", user_id="test"):
    print(event)

## If you are using Cloud Run

#### Remote Testing

For more information about authenticating HTTPS requests to Cloud Run services, see:
[Cloud Run Authentication Documentation](https://cloud.google.com/run/docs/triggering/https-request)

Remote testing involves using a deployed service URL instead of localhost.

Authentication is handled using GCP identity tokens instead of local credentials.

In [ ]:
ID_TOKEN = get_ipython().getoutput("gcloud auth print-identity-token -q")[0]

In [ ]:
SERVICE_URL = "YOUR_SERVICE_URL_HERE"  # Replace with your Cloud Run service URL

Send a message

In [ ]:
# Create A2A message request
message = Message(
    message_id=f"msg-user-{uuid.uuid4()}",
    role=Role.user,
    parts=[Part(root=TextPart(text="Hello! Weather in New York?"))],
)

request = SendStreamingMessageRequest(
    id=f"req-{uuid.uuid4()}",
    params=MessageSendParams(message=message),
)

# Set up headers with authentication
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {ID_TOKEN}"}

# Send the streaming request to the A2A endpoint
response = requests.post(
    f"{SERVICE_URL}/a2a/app",
    headers=headers,
    json=request.model_dump(mode="json", exclude_none=True),
    stream=True,
    timeout=60,
)

print(f"Response status code: {response.status_code}")

# Parse streaming A2A responses
for line in response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")

#### Local Testing

> You can run the application locally via the `make local-backend` command.

Send a message

In [ ]:
# Create A2A message request
message = Message(
    message_id=f"msg-user-{uuid.uuid4()}",
    role=Role.user,
    parts=[Part(root=TextPart(text="Hello! Weather in New York?"))],
)

request = SendStreamingMessageRequest(
    id=f"req-{uuid.uuid4()}",
    params=MessageSendParams(message=message),
)

# Set up headers
headers = {"Content-Type": "application/json"}

# Send the streaming request to the local A2A endpoint
response = requests.post(
    "http://127.0.0.1:8000/a2a/app",
    headers=headers,
    json=request.model_dump(mode="json", exclude_none=True),
    stream=True,
    timeout=60,
)

print(f"Response status code: {response.status_code}")

# Parse streaming A2A responses
for line in response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")